# CH 16 - Coroutines

## TOC<a id='toc'></a>
* [Ch16 Notes](#ch16_notes)

### CH16 Notes <a id='ch16_notes'></a>
[toc](#toc)
### Coroutines

* coroutines are meant to implement *cooperatove multitasking*
    - each coroutin yields control to a central scheduler so that other coroutines can be activated
* Coroutines is syntatically like a generator (function): just a function with a `yield` keyword in the body. 
* However in a coroutine, the yield usually appears on the right hand side of an expression and it may or may not produce a value (ie have a value after the yield)
* the coroutine may receive data from the caller via `.send()` (also available are `.throw()` and `.close()`)

* simple example:

In [1]:
def simple_coro(a):
    print('-> Started: a = ', a)
    b = yield a
    print('-> Received: b = ', b)
    c = yield a + b
    print('-> Received: c = ', c)

In [2]:
my_coro = simple_coro(14)

In [3]:
import inspect

In [4]:
inspect.getgeneratorstate(my_coro)

'GEN_CREATED'

In [6]:
next(my_coro)

-> Started: a =  14


14

In [7]:
my_coro.send(28)

-> Received: b =  28


42

In [8]:
next(my_coro)

-> Received: c =  None


StopIteration: 

* VIP: execution of corouting is suspended exactly at yield - so assignment doesn't happen till after it resume execution.
    - and the assigned value is NOT rhs of line, but rather what gets sent via .send()
    - this is extremely weird looking at first

* a coroutine can be in one of 4 states:
    - GEN_CREATED: waiting to start execution; can't receive values via send
        - so after creation, must use next(my_gen) to **prime** it
        - can create a decorator to take care of this.
    - GEN_RUNNING: currenlty being executed by interpreter (will only see this in multithreaded applications)
    - GEN_SUSPENDED: Currently suspended at a yield expression; can receive values
    - GEN_CLOSED: execution has completed

### Coroutine termination and exception Handling
* Unhandle exception inside corouting terminates its execution, and the exception propagates to caller
    - this used to be taken advantage off to terminate execution of coroutine
    - people used to pass singletons like `Ellipsis`, which almost never occurs in data streams, so typically not handled by code.
* since python 2.5, now have `.throw()` and `.close()`
    - `generator.throw(exc_type[, exc_value[, traceback]])` - causes yield expression where gen was paused to raise exception
        - if handled by gen, flow advances to next yield, and value yielded becomes result of throw() call
        - if not handled, exception propagates to context of caller
    - `generator.close()` - causes yield expression to raise a `GeneratorExit` exception
        - No error reported to caller if exeption not handled by gen, or if gen raises StopIteration (usually by running to completion)
        - if value is yielded after this, RuntimeError is raised which propagates to caller
* If it is necesarry to do some cleanup no matter how coroutine ends, you need to wrap it in a try/finally block
* can return values at end of coroutine
    - the value attribute of the StopIteration exception contains the returned data
    - syntax error before python 3.3
    - `yield from` catches StopIteratrion internally, and is transparent to user (like for loop)

### Using Yield From

* `yield from` is a new language construct
    - similar constructs in other languages are called `await`, and that would be a much better name.
* when a gen calls yield from subgen(), the subgen() takes over and will yield values *to the caller* fo the gen; meanwhile the gen will be blocked, waiting until subgen terminates.
* first thing yield from x does is call iter(x), so it can be called on any iterable - but it really shines when x is generator.
* The main feature of yield from is that it opens a bidirectional channel from the outermost caller, to the innermost subgenerator so that values can be sent and yielded back and forth directly from them, and exceptions can be thrown all the way in without adding a lot of boiler plate in intermediate coroutines.
* THE MAIN POINT: values yielded from subgenerator, are not yielded via delegating generator, and therefore delegating generator code doesn't continue to run - until subgenerator is done; then return value actually goes back to delegating generator.

In [13]:
def subg(x):
    count = 0
    while count < x:
        yield count
        count +=1
    return count

def delg(y):
    count = 0
    while count < y:
        a = yield from subg(y-count)
        count+=1
        print('del gen loop. a: ', a)

In [14]:
gen1 = subg(10)

In [15]:
list(gen1)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [16]:
gen2 = delg(5)

In [17]:
list(gen2)

del gen loop. a:  5
del gen loop. a:  4
del gen loop. a:  3
del gen loop. a:  2
del gen loop. a:  1


[0, 1, 2, 3, 4, 0, 1, 2, 3, 0, 1, 2, 0, 1, 0]

In [33]:
def averager():
    total, count, average = 0.0,0, None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count +=1
    return (count, total/count)

In [34]:
avg = averager()

In [35]:
next(avg)

In [36]:
avg.send(1)

In [37]:
avg.send(2)

In [38]:
avg.send(None)

StopIteration: (2, 1.5)

I DONT UNDERSTAND THE NEED FOR THE WHILE LOOP IN THE FOLLOWING CODE

In [39]:
def grouper(result, key):
    while True:
        result[key] = yield from averager()
        
# This does not work.
# def grouper(result, key):
#     try:
#         result[key] = yield from averager()
#     except StopIteration:
#         return 3

I thin the answer is that the subgen StopIteration is not propagated up. So it doesn't raise, it just returns. But If we allow delegating to finish, it will raise StopIteration, and we have to catch that in the caller.

In [44]:
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)
    print(results)

In [45]:
data = {'a':[1,2,3], 'b':[4,5,6]}

In [46]:
main(data)

{'a': (3, 2.0), 'b': (3, 5.0)}


VIP: if a subgenerator never terminates, the delegating generator will be suspended forever at the yield from. This will not prevent your program from making progress, because yield from transfers control to client, but it does mean some tasks will remain unfinished.
    - in the case above, without send None, previous group would be garbage collected when new one created.

* Yield from allows you to 'pipe' different coroutines together
* can pipe more than two
* eventually must terminate in a yield, or any iterable object
* (asside: sending a value also triggers a next - so it yield something if there is somethign to yield)

### rules:
* any value subgenerator yields passed directly to caller of delegating gen
* values sent to delegating gen using send() are passed directly to subgen. If call raises StopIteration, then the delgating generator is resumed. Any other exception is propagated to the delegating generator.
* return expr in a gen or subgen causes StopIteration(expr) to be raised upon exit from gen.
* the value of the yield from expression is the first argument to the stop iteration exception raised by the subgenerator when it terminates
* exceptions other that GeneratorExit thrown into the delegating generator are passed to throw() of the subgenerator. If sub raises StopIteration, delegating resumes. Anything else propagated to delegating.
* If GeneratorExit is throw into delegating, or close() of delegating called, then close() of sub called, it if has one. If this results in eception, it is propagated into delegating. Othewise GeneratorExit raised in delegating.

In [53]:
def averager():
    total, count, average = 0.0,0, None
    while True:
        term = yield count
        if term is None:
            break
        total += term
        count +=1
    return (count, total/count)

In [54]:
def grouper(result, key):
    while True:
        result[key] = yield from averager()

In [55]:
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            res = group.send(value)
            print(res)
        group.send(None)
    print(results)

In [56]:
data = {'a':[1,2,3], 'b':[4,5,6]}

In [57]:
main(data)

1
2
3
1
2
3
{'a': (3, 2.0), 'b': (3, 5.0)}


## Use case: Coroutines for Discrete Event Simulation (DES)
* *DES* time doesn't advance at constat rate - it advance till time of next discrete event. [in contrast to a continuous simulation]
    - turn based games are exmaples of a DES
* corroutines offer precisely the right abstraction for writing a DES
* He creates a taxi pickup/dropoff simulation using a dictionary of taxi generators, and a priority queue
    * priority queues are a fundamental building blocl of discrete event simulators: events are created in any order, place in the queue, and later retrievec in order  according to the schedule time of each one.
    * `queue.PriorityQueue()`

### Hot off the press

* **async** and **await** keywords have been added to python to better distinguish coroutines and generators